<a href="https://colab.research.google.com/github/WanjohiWanjohi/BCG_Forage_/blob/main/BCG_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [4]:
training_df  = pd.read_csv('ml_case_training_data.csv')
training_history = pd.read_csv('ml_case_training_hist_data.csv')
training_output = pd.read_csv('ml_case_training_output.csv')

In [54]:
training_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16096 entries, 0 to 16095
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        16096 non-null  object 
 1   activity_new              6551 non-null   object 
 2   channel_sales             11878 non-null  object 
 3   cons_12m                  16096 non-null  int64  
 4   cons_gas_12m              16096 non-null  int64  
 5   cons_last_month           16096 non-null  int64  
 6   date_activ                16096 non-null  object 
 7   date_end                  16094 non-null  object 
 8   date_first_activ          3508 non-null   object 
 9   date_modif_prod           15939 non-null  object 
 10  date_renewal              16056 non-null  object 
 11  forecast_base_bill_ele    3508 non-null   float64
 12  forecast_base_bill_year   3508 non-null   float64
 13  forecast_bill_12m         3508 non-null   float64
 14  foreca

In [22]:
training_history.info() , training_history.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172367 entries, 0 to 172366
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            172367 non-null  object 
 1   price_date    172367 non-null  object 
 2   price_p1_var  171155 non-null  float64
 3   price_p2_var  171155 non-null  float64
 4   price_p3_var  171155 non-null  float64
 5   price_p1_fix  171155 non-null  float64
 6   price_p2_fix  171155 non-null  float64
 7   price_p3_fix  171155 non-null  float64
dtypes: float64(6), object(2)
memory usage: 10.5+ MB


(None, id               object
 price_date       object
 price_p1_var    float64
 price_p2_var    float64
 price_p3_var    float64
 price_p1_fix    float64
 price_p2_fix    float64
 price_p3_fix    float64
 dtype: object)

In [23]:
training_output.info() , training_output.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16096 entries, 0 to 16095
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      16096 non-null  object
 1   churn   16096 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 251.6+ KB


(None, id       object
 churn     int64
 dtype: object)

From the info on the datasets , we can see the campaign disc_ele is blank  fully , so we can drop this

In [24]:
training_df.drop(inplace=True , columns=['campaign_disc_ele'])

In [53]:
training_output[pd.to_numeric(training_output.id, errors='coerce').isnull()]

,id,churn
0,48ada52261e7cf58715202705a0451c9,0
1,24011ae4ebbe3035111d65fa7c15bc57,1
2,d29c2c54acc38ff3c0614d0a653813dd,0
3,764c75f661154dac3a6c254cd082ea7d,0
4,bba03439a292a1e166f80264c16191cb,0
...,...,...
16091,18463073fb097fc0ac5d3e040f356987,0
16092,d0a6f71671571ed83b2645d23af6de00,1
16093,10e6828ddd62cbcf687cb74928c4c2d2,1
16094,1cf20fd6206d7678d5bcafd28c53b4db,0


In [62]:
#missing values 

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed_train = imp_mean.fit_transform(training_df)
imputed_train_df = pd.DataFrame(imputed_train)

In [ ]:
#make a simple encoding and data transformation 


In [60]:
#calculate collinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [64]:
coll = calc_vif(imputed_train_df)

TypeError: ignored

In [58]:
#join the output and training dfs into one off of the id that contain customer inforamtion


In [59]:
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,48ada52261e7cf58715202705a0451c9,esoiiifxdlbkcsluxmfuacbdckommixw,lmkebamcaaclubfxadlmueccxoimlema,309275,0,10025,2012-11-07,2016-11-06,2009-07-14,2012-11-07,2015-11-09,0,0,0,0,26520.3,10025,0,359.29,0.095919,0.088347,58.996,f,831.8,-41.76,-41.76,1,1732.36,3,ldkssxwpmemidmecebumciepifcamkci,180
1,24011ae4ebbe3035111d65fa7c15bc57,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2009-07-14,2015-11-01,2015-06-23,0,0,0,0,0,0,0,1.78,0.114481,0.098142,40.6067,t,0,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648
2,d29c2c54acc38ff3c0614d0a653813dd,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,4660,0,0,2009-08-21,2016-08-30,2009-07-14,2009-08-21,2015-08-31,0,0,0,0,189.95,0,0,16.27,0.145711,0,44.3114,f,0,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.8
3,764c75f661154dac3a6c254cd082ea7d,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2009-07-14,2010-04-16,2015-04-17,0,0,0,0,47.96,0,0,38.72,0.165794,0.087899,44.3114,f,0,28.6,28.6,1,6.6,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856
4,bba03439a292a1e166f80264c16191cb,apdekpcbwosbxepsfxclislboipuxpop,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2009-07-14,2010-03-30,2015-03-31,0,0,0,0,240.04,0,0,19.83,0.146694,0,44.3114,f,0,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,18463073fb097fc0ac5d3e040f356987,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,2012-05-24,2016-05-08,2009-07-14,2015-05-08,2014-05-26,0,0,0,0,4648.01,0,0,18.57,0.138305,0,44.3114,t,0,27.88,27.88,2,381.77,4,lxidpiddsbxsbosboudacockeimpuepw,15
16092,d0a6f71671571ed83b2645d23af6de00,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,2012-08-27,2016-08-27,2012-08-27,2012-08-27,2015-08-28,68.64,68.64,1254.65,15.94,631.69,181,0,144.03,0.100167,0.091892,58.996,f,15.94,0,0,1,90.34,3,lxidpiddsbxsbosboudacockeimpuepw,6
16093,10e6828ddd62cbcf687cb74928c4c2d2,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,2012-02-08,2016-02-07,2009-07-14,2012-02-08,2015-02-09,0,0,0,0,190.39,179,0,129.6,0.1169,0.100015,40.6067,f,18.05,39.84,39.84,1,20.38,4,lxidpiddsbxsbosboudacockeimpuepw,15.935
16094,1cf20fd6206d7678d5bcafd28c53b4db,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,131,0,0,2012-08-30,2016-08-30,2009-07-14,2012-08-30,2015-08-31,0,0,0,0,19.34,0,0,7.18,0.145711,0,44.3114,f,0,13.08,13.08,1,0.96,3,lxidpiddsbxsbosboudacockeimpuepw,11
